# Performance Validation

This notebook validates the performance metrics implemented in `src/backtest_data_module/backtesting/performance.py`.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, skew, kurtosis
import sys
sys.path.append('../src')
from backtest_data_module.backtesting.performance import Performance

## 1. Generate Synthetic Returns Data

In [ ]:
np.random.seed(42)
nav_series = np.random.normal(loc=0.0005, scale=0.01, size=1000).cumsum() + 100

## 2. Calculate Performance Metrics

In [ ]:
perf = Performance(nav_series=nav_series.tolist())
metrics = perf.compute_metrics()
print(metrics.to_json())

## 3. Reference Implementation

We will compare our implementation with a well-known library like `empyrical`.

In [ ]:
try:
    import empyrical as ep
    returns = pd.Series(perf.returns)
    print("Sharpe Ratio (empyrical):", ep.sharpe_ratio(returns))
    print("Sortino Ratio (empyrical):", ep.sortino_ratio(returns))
    print("Max Drawdown (empyrical):", ep.max_drawdown(returns))
except ImportError:
    print("empyrical not installed. Skipping comparison.")

## 4. CPCV Logic Flowchart

```mermaid
graph TD
    A[Start] --> B{Partition Data into N Groups};
    B --> C{Select K Combinations of Test Sets};
    C --> D{For each combination};
    D --> E[Purge Overlapping Data];
    E --> F[Apply Embargo];
    F --> G[Train Strategy];
    G --> H[Test Strategy];
    H --> I[Compute Performance Metrics];
    I --> D;
    D --> J[Aggregate Results];
    J --> K[End];
```